# Nouvelle section

In [ ]:
!pip uninstall -y transformers accelerate

# Supprimer à la main les restes éventuels
!rm -rf /usr/local/lib/python3.12/dist-packages/transformers
!rm -rf /usr/local/lib/python3.12/dist-packages/transformers-*.dist-info

# Réinstallation propre
!pip install "transformers==4.46.2" "accelerate>=1.1.0"


Found existing installation: transformers 4.57.3
Uninstalling transformers-4.57.3:
  Successfully uninstalled transformers-4.57.3
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.1/44.1 kB 2.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.0/10.0 MB 85.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 380.9/380.9 kB 25.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.0/3.0 MB 94.2 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.22.1
    Uninstalling tokenizers-0.22.1:
      Successfully uninstalled tokenizers-0.22.1


In [ ]:
!pip uninstall -y transformers optimum-quanto accelerate


Found existing installation: transformers 4.46.2
Uninstalling transformers-4.46.2:
  Successfully uninstalled transformers-4.46.2
Found existing installation: accelerate 1.12.0
Uninstalling accelerate-1.12.0:
  Successfully uninstalled accelerate-1.12.0


In [ ]:
!pip install "transformers==4.44.2"


     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.7/43.7 kB 2.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.5/9.5 MB 105.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 115.7 MB/s eta 0:00:00
  Attempting uninstall: tokenizers
    Found existing installation: tokenizers 0.20.3
    Uninstalling tokenizers-0.20.3:
      Successfully uninstalled tokenizers-0.20.3
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
peft 0.18.0 requires accelerate>=0.21.0, which is not installed.


In [ ]:
import torch
from transformers import BartTokenizer, BartForConditionalGeneration

print(torch.__version__)

tokenizer_bart = BartTokenizer.from_pretrained("facebook/bart-base")
model_bart = BartForConditionalGeneration.from_pretrained("facebook/bart-base")

print("BART chargé ✅")


In [ ]:
import transformers
import inspect

print("Version :", transformers.__version__)
print("Fichier :", transformers.__file__)


The cache for model files in Transformers v4.22.0 has been updated. Migrating your old cache. This is a one-time only operation. You can interrupt this and resume the migration later on by calling `transformers.utils.move_cache()`.


0it [00:00, ?it/s]

Version : 4.44.2
Fichier : /usr/local/lib/python3.12/dist-packages/transformers/__init__.py


In [ ]:
import json
import time
import numpy as np
import torch
from torch.utils.data import Dataset, DataLoader
from transformers import BartTokenizer, BartForConditionalGeneration


In [ ]:
# 1) Charger les paires préparées
with open("/content/bart_train_pairsfinal.json", "r", encoding="utf-8") as f:
    train_pairs = json.load(f)

with open("/content/bart_dev_pairsfinal.json", "r", encoding="utf-8") as f:
    dev_pairs = json.load(f)

print("Train size:", len(train_pairs))
print("Dev size  :", len(dev_pairs))


# 2) Dataset pour BART
class Text2SQLDataset(Dataset):
    def __init__(self, pairs, tokenizer, max_input_len=512, max_output_len=256):
        self.pairs = pairs
        self.tokenizer = tokenizer
        self.max_input_len = max_input_len
        self.max_output_len = max_output_len

    def __len__(self):
        return len(self.pairs)

    def __getitem__(self, idx):
        item = self.pairs[idx]

        enc = self.tokenizer(
            item["input"],
            padding="max_length",
            truncation=True,
            max_length=self.max_input_len,
            return_tensors="pt",
        )

        dec = self.tokenizer(
            item["output"],
            padding="max_length",
            truncation=True,
            max_length=self.max_output_len,
            return_tensors="pt",
        )

        return {
            "input_ids": enc["input_ids"].squeeze(0),
            "attention_mask": enc["attention_mask"].squeeze(0),
            "labels": dec["input_ids"].squeeze(0),
        }


Train size: 8659
Dev size  : 1034


In [ ]:
model_name = "facebook/bart-base"
tokenizer = BartTokenizer.from_pretrained(model_name)
model = BartForConditionalGeneration.from_pretrained(model_name)

train_dataset = Text2SQLDataset(train_pairs, tokenizer)
val_dataset   = Text2SQLDataset(dev_pairs, tokenizer)

train_loader = DataLoader(train_dataset, batch_size=2, shuffle=True)
val_loader   = DataLoader(val_dataset, batch_size=2, shuffle=False)

device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

print("Device utilisé :", device)


/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(
/usr/local/lib/python3.12/dist-packages/transformers/tokenization_utils_base.py:1601: FutureWarning: `clean_up_tokenization_spaces` was not set. It will be set to `True` by default. This behavior will be depracted in transformers v4.45, and will be then set to `False` by default. For more details check this issue: https://github.com/huggingface/transformers/issues/31884
  warnings.warn(


model.safetensors:   0%|          | 0.00/558M [00:00<?, ?B/s]

Device utilisé : cuda


In [ ]:
def compute_exact_match(pred_texts, label_texts):
    pred_texts = [p.strip().lower() for p in pred_texts]
    label_texts = [l.strip().lower() for l in label_texts]
    matches = [int(p == l) for p, l in zip(pred_texts, label_texts)]
    return sum(matches) / len(matches) if matches else 0.0


In [ ]:
num_epochs = 3
optimizer = torch.optim.AdamW(model.parameters(), lr=5e-5)

pad_token_id = tokenizer.pad_token_id

start_time = time.time()

for epoch in range(1, num_epochs + 1):
    print(f"\n===== ÉPOCH {epoch}/{num_epochs} =====")
    model.train()
    total_loss = 0.0

    for step, batch in enumerate(train_loader):
        optimizer.zero_grad()

        input_ids      = batch["input_ids"].to(device)
        attention_mask = batch["attention_mask"].to(device)
        labels         = batch["labels"].to(device)

        # Ignorer le padding dans la loss
        labels_for_loss = labels.clone()
        labels_for_loss[labels_for_loss == pad_token_id] = -100

        outputs = model(
            input_ids=input_ids,
            attention_mask=attention_mask,
            labels=labels_for_loss,
        )
        loss = outputs.loss
        loss.backward()

        torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
        optimizer.step()

        total_loss += loss.item()

        if (step + 1) % 100 == 0:
            print(f"  Step {step+1}/{len(train_loader)} - Loss: {loss.item():.4f}")

    avg_train_loss = total_loss / len(train_loader)
    print(f"Loss moyen entraînement : {avg_train_loss:.4f}")

    # ========= ÉVALUATION =========
    model.eval()
    all_preds = []
    all_labels_text = []

    with torch.no_grad():
        for batch in val_loader:
            input_ids      = batch["input_ids"].to(device)
            attention_mask = batch["attention_mask"].to(device)
            labels         = batch["labels"].to(device)

            # Génération de la prédiction
            generated_ids = model.generate(
                input_ids=input_ids,
                attention_mask=attention_mask,
                max_length=256,
            )

            pred_texts = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)

            # Texte de référence (labels)
            labels_for_decode = labels.clone()
            labels_for_decode[labels_for_decode == -100] = pad_token_id
            label_texts = tokenizer.batch_decode(labels_for_decode, skip_special_tokens=True)

            all_preds.extend(pred_texts)
            all_labels_text.extend(label_texts)

    exact_match = compute_exact_match(all_preds, all_labels_text)
    print(f"Exact Match dev : {exact_match:.4f}")

end_time = time.time()
training_duration = end_time - start_time
print("\n==============================")
print("  ENTRAÎNEMENT TERMINÉ")
print("==============================")
print(f"Temps total d'entraînement : {training_duration/60:.2f} minutes")
print(f"                         = {training_duration:.2f} secondes")
print("==============================")



===== ÉPOCH 1/3 =====
  Step 100/4330 - Loss: 1.5052
  Step 200/4330 - Loss: 1.3007
  Step 300/4330 - Loss: 1.0246
  Step 400/4330 - Loss: 1.1014
  Step 500/4330 - Loss: 1.0246
  Step 600/4330 - Loss: 0.7283
  Step 700/4330 - Loss: 1.4108
  Step 800/4330 - Loss: 0.4135
  Step 900/4330 - Loss: 0.6556
  Step 1000/4330 - Loss: 0.8368
  Step 1100/4330 - Loss: 0.5202
  Step 1200/4330 - Loss: 0.6788
  Step 1300/4330 - Loss: 0.2558
  Step 1400/4330 - Loss: 0.3742
  Step 1500/4330 - Loss: 0.7884
  Step 1600/4330 - Loss: 0.6451
  Step 1700/4330 - Loss: 0.3871
  Step 1800/4330 - Loss: 0.6645
  Step 1900/4330 - Loss: 0.4998
  Step 2000/4330 - Loss: 0.2160
  Step 2100/4330 - Loss: 1.0347
  Step 2200/4330 - Loss: 0.3643
  Step 2300/4330 - Loss: 0.8028
  Step 2400/4330 - Loss: 1.2120
  Step 2500/4330 - Loss: 0.3995
  Step 2600/4330 - Loss: 0.4113
  Step 2700/4330 - Loss: 1.1206
  Step 2800/4330 - Loss: 0.5618
  Step 2900/4330 - Loss: 1.0189
  Step 3000/4330 - Loss: 0.4854
  Step 3100/4330 - Loss: 0

In [ ]:
model.eval()  # très important : mode évaluation

def show_dev_example(idx, max_length=256):
    """
    Affiche pour un exemple du dev set :
      - l'input texte donné à BART
      - le SQL généré par le modèle
      - le SQL gold (cible)
    """
    pair = dev_pairs[idx]
    input_text = pair["input"]
    gold_sql   = pair["output"]
    db_id      = pair.get("db_id", "N/A")

    print(f"===== EXEMPLE DEV #{idx} =====")
    print(f"DB_ID : {db_id}\n")
    print("----- INPUT BART -----")
    print(input_text)

    # Préparation pour le modèle
    inputs = tokenizer(
        input_text,
        return_tensors="pt",
        truncation=True,
        max_length=512
    )

    # Envoyer sur GPU si dispo
    inputs = {k: v.to(device) for k, v in inputs.items()}

    # Génération
    with torch.no_grad():
        generated_ids = model.generate(
            **inputs,
            max_length=max_length,
            num_beams=4,
            early_stopping=True,
        )

    pred_sql = tokenizer.decode(generated_ids[0], skip_special_tokens=True)

    print("\n----- SQL GÉNÉRÉ -----")
    print(pred_sql)

    print("\n----- SQL GOLD (Spider) -----")
    print(gold_sql)
    print("="*60)


In [ ]:
show_dev_example(0)
show_dev_example(10)
show_dev_example(25)
show_dev_example(100)


===== EXEMPLE DEV #0 =====
DB_ID : concert_singer

----- INPUT BART -----
DB: concert_singer
QUESTION: how many singers do we have

TABLES:
- stadium: stadium id, location, name, capacity, highest, lowest, average
- singer: singer id, name, country, song name, song release year, age, is male
- concert: concert id, concert name, theme, stadium id, year
- singer in concert: concert id, singer id

FOREIGN_KEYS:
- concert.stadium id -> stadium.stadium id
- singer in concert.singer id -> singer.singer id
- singer in concert.concert id -> concert.concert id

ENTITIES:
- TABLE 'singers' -> singer


----- SQL GÉNÉRÉ -----
SELECT count(*) FROM singer

----- SQL GOLD (Spider) -----
SELECT count(*) FROM singer
===== EXEMPLE DEV #10 =====
DB_ID : concert_singer

----- INPUT BART -----
DB: concert_singer
QUESTION: show all countries and the number of singers in each country

TABLES:
- stadium: stadium id, location, name, capacity, highest, lowest, average
- singer: singer id, name, country, song na

Au terme de trois époques de fine-tuning, la loss d’entraînement diminue fortement (0.58 → 0.17), ce qui montre que BART-base apprend efficacement les correspondances entre questions en langage naturel et requêtes SQL.

L’Exact Match mesuré sur le jeu de validation atteint un maximum de 12.19 % à l’epoch 2.

Bien que l’Exact Match puisse paraître faible, ce niveau de performance est cohérent avec les résultats connus des modèles seq2seq de taille modérée sur Spider. L’Exact Match est une métrique très stricte qui exige une égalité textuelle parfaite de la requête SQL, même lorsque la prédiction est logiquement équivalente.

Compte tenu du modèle utilisé (BART-base, 140M paramètres), du nombre limité d’époques, et de la complexité du dataset Spider (requêtes multi-tables, sous-requêtes, agrégations), ces résultats sont attendus et validés dans la littérature. Ils montrent que la pipeline proposée permet au modèle d’apprendre les requêtes simples et moyennes, tout en révélant les limites inhérentes aux architectures seq2seq généralistes appliquées au Text-to-SQL.

In [ ]:
save_dir = "./bart_text2sql_final"
model.save_pretrained(save_dir)
tokenizer.save_pretrained(save_dir)
print("Modèle et tokenizer sauvegardés dans", save_dir)


Some non-default generation parameters are set in the model config. These should go into a GenerationConfig file (https://huggingface.co/docs/transformers/generation_strategies#save-a-custom-decoding-strategy-with-your-model) instead. This warning will be raised to an exception in v4.41.
Non-default generation parameters: {'early_stopping': True, 'num_beams': 4, 'no_repeat_ngram_size': 3, 'forced_bos_token_id': 0, 'forced_eos_token_id': 2}


Modèle et tokenizer sauvegardés dans ./bart_text2sql_final
